# Worksheet 07

Name:  Zhengxu Wang  
UID: U81066856

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.5748006736896984, 5.22017210623626, 6.052945758152454, 4.766009824263725, 6.460686352971145, 4.093002571237734, 4.64187016682607, 3.810158202666821, 4.201807099416523, 5.450693033707913]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
mean, stdev = 8, 1
c2 = np.random.normal( mean , stdev , 10 ).tolist()
print(c2)

[6.5457936171092665, 9.347198536249596, 7.852211386850938, 7.264315336111502, 9.173653674651003, 8.796941397022959, 7.411845946032614, 7.671464387804637, 9.105995451013433, 7.983190961684002]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[5.450693033707913, 4.201807099416523, 3.810158202666821, 7.983190961684002, 9.105995451013433, 7.671464387804637, 7.411845946032614, 8.796941397022959, 9.173653674651003, 7.264315336111502]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

    mean,stdev  
c1 : 5, 1  
c2 : 8, 1  

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [len(c1) / (len(c1) + len(c2)), len(c2) / (len(c1) + len(c2))]
mean = [sum(c1)/len(c1), sum(c2)/len(c2)]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.983190961684002, 9.105995451013433, 7.671464387804637, 7.411845946032614, 8.796941397022959, 9.173653674651003, 7.264315336111502]
[5.450693033707913, 4.201807099416523, 3.810158202666821]
P(C_1) = 0.7,  P(C_2) = 0.3
mean_1 = 8.20105816490288,  mean_2 = 4.4875527785970855
var_1 = 0.564029869197574,  var_2 = 0.4893843852283513


It is close to the true values.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  5.450693033707913
probability of observing that point if it came from cluster 0 =  4.855867969867283e-06
probability of observing that point if it came from cluster 1 =  0.11754061796403166
point =  4.201807099416523
probability of observing that point if it came from cluster 0 =  8.560595422357825e-12
probability of observing that point if it came from cluster 1 =  0.6874308006613035
point =  3.810158202666821
probability of observing that point if it came from cluster 0 =  4.892923447891249e-14
probability of observing that point if it came from cluster 1 =  0.31276461956040574
point =  7.983190961684002
probability of observing that point if it came from cluster 0 =  0.6564607986902339
probability of observing that point if it came from cluster 1 =  6.793514381708171e-12
point =  9.105995451013433
probability of observing that point if it came from cluster 0 =  0.19527192027498866
probability of observing that point if it came from cluster 1 =  3.73044987990739e-20
point = 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_c0_x) / len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum([prob_c0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_c0_x), sum([prob_c1_x[i] * (data[i] - mean[1])**2 for i in range(len(data))]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.7000096161976568,  P(C_2) = 0.2999903838023431
mean_1 = 8.20102032392221,  mean_2 = 4.487522041773716
var_1 = 0.5641262499200231,  var_2 = 0.4893708494511041


Only have slighty differences

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
from scipy.stats import norm

prob_c0_x = []
prob_c1_x = []

for p in data:
    pdf_i = [norm.pdf(p, mean[j], var[j]) for j in range(k)]
    prob_x = sum(prob_c[j] * pdf_i[j] for j in range(k))

    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)

for datapoint, prob_0, prob_1 in probs:
    print(datapoint)
    print("Probability of coming from C_1 = " + str(prob_0))
    print("Probability of coming from C_2 = " + str(prob_1))
    print()


5.450693033707913
Probability of coming from C_1 = 9.681746536963193e-05
Probability of coming from C_2 = 0.9999031825346304

4.201807099416523
Probability of coming from C_1 = 2.931636900080027e-11
Probability of coming from C_2 = 0.9999999999706837

3.810158202666821
Probability of coming from C_1 = 3.6895181756902443e-13
Probability of coming from C_2 = 0.999999999999631

7.983190961684002
Probability of coming from C_1 = 0.9999999999955727
Probability of coming from C_2 = 4.427363517366729e-12

9.105995451013433
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 8.160925557434552e-20

7.671464387804637
Probability of coming from C_1 = 0.9999999995066877
Probability of coming from C_2 = 4.933123281007401e-10

7.411845946032614
Probability of coming from C_1 = 0.9999999768451137
Probability of coming from C_2 = 2.315488630812912e-08

8.796941397022959
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 1.2504934197701962e-17

9.173653674651003
Pro

There are some probability nearly reach 1, that means these points been classified in a high confidence interval. Which is not happend in kmeans.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [9]:
labels = []

for i in range(len(data)):
    if prob_c0_x[i] > prob_c1_x[i]:
        labels.append(0)
    else:
        labels.append(1)

print(labels)

[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      Dis       |
| A  C |      Agree       |
| A  D |      Agree       |
| A  E |      Agree       |
| B  C |      Dis       |
| B  D |      Agree       |
| B  E |      Agree       |
| C  D |      Agree       |
| C  E |      Agree       |
| D  E |      Dis       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

nC2 = n! / (2! * (n-2)!)

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

nC2 = (n * (n-1)) / 2 = (9 * 8) / 2 = 36

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

In cluster 0, there are (3 x 2) / 2 = 3 pairs that agree with C.  
In cluster 1, there are (2 x 1) / 2 = 1 pairs that agree with C.  
In cluster 2, there are (4 x 3) / 2 = 6 pairs that agree with C.  
36 - 10 = 26

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

cluster it using a clustering algorithm (e.g., k-means) and return the cluster labels.

Once the clustering of each subset is complete, merge the resulting clusters together to form larger clusters.

For each pair of clusters, compute the number of disagreements between them using the formula from part (b).

Continue merging clusters until there are no more clusters left.

The total number of disagreements between the original clusters is the sum of the disagreements between all pairs of clusters.